In [52]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from string import punctuation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, concatenate
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy import sparse as sp

In [15]:
df = pd.read_csv("D:\\Private\\University\\Research\\Data Sets\\all_tickets.csv")
df

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4
...,...,...,...,...,...,...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3


In [16]:
value_counts = df['impact'].value_counts()
value_counts

impact
4    34621
3    13184
0      471
2      228
1       45
Name: count, dtype: int64

In [9]:
df.dropna(subset=['title', 'body', 'impact'], inplace=True)
print(df.isnull().sum())

title               0
body                0
ticket_type         0
category            0
sub_category1       0
sub_category2       0
business_service    0
urgency             0
impact              0
dtype: int64


In [10]:
value_counts = df['impact'].value_counts()
value_counts

impact
4    34033
3    13068
0      465
2      227
1       44
Name: count, dtype: int64

In [11]:
# Define the replacement dictionary
replacement_dict = {0: 1, 1: 1, 2: 1, 3: 2, 4: 3}

# Map the values in the 'impact' column using the replacement dictionary
df['impact'] = df['impact'].map(replacement_dict)

# Verify the updated counts
print(df['impact'].value_counts())

impact
3    34033
2    13068
1      736
Name: count, dtype: int64


In [19]:
df = df.dropna(subset=['body', 'impact', 'title'])
nltk.download('stopwords')
nltk.download('punkt')
df['description'] = df['body']
df

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\HP\AppData\Local\Temp\ipykernel_22960\3605154046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['body']


,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,description
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,icon dear please setup icon per icon engineers...
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,work experience user hi work experience studen...
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,requesting meeting hi please help follow equip...
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,re expire days hi ask help update passwords co...
5,mail,verification warning hi has got attached pleas...,1,4,3,7,89,3,4,verification warning hi has got attached pleas...
...,...,...,...,...,...,...,...,...,...,...
48544,git space for a project issues with adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3,sent tuesday july re git space for hi got righ...
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3,sent july error hi guys can you help out with ...
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3,sent tuesday july connection issues hello have...
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3,sent tuesday july error hello guys have receiv...


In [20]:
def preprosessor(text):
    tokens = word_tokenize(text.lower())
    stop_words = set(stopwords.words('english') + list(punctuation))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    preprocessed_text = ' '.join(filtered_tokens)
    return preprocessed_text

df['description'] = df['description'].apply(preprosessor)
df['title'] = df['title'].apply(preprosessor)
df

C:\Users\HP\AppData\Local\Temp\ipykernel_22960\2567923999.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['description'] = df['description'].apply(preprosessor)
C:\Users\HP\AppData\Local\Temp\ipykernel_22960\2567923999.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'] = df['title'].apply(preprosessor)


,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact,description
1,connection icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4,icon dear please setup icon per icon engineers...
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4,work experience user hi work experience studen...
3,requesting meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4,requesting meeting hi please help follow equip...
4,reset passwords external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4,expire days hi ask help update passwords colle...
5,mail,verification warning hi has got attached pleas...,1,4,3,7,89,3,4,verification warning hi got attached please ad...
...,...,...,...,...,...,...,...,...,...,...
48544,git space project issues adding users,sent tuesday july re git space for hi got righ...,0,4,21,1,39,1,3,sent tuesday july git space hi got rights prob...
48545,error,sent july error hi guys can you help out with ...,0,4,9,41,10,2,3,sent july error hi guys help error appearing w...
48546,connection issues,sent tuesday july connection issues hello have...,0,5,32,0,32,2,3,sent tuesday july connection issues hello conn...
48547,error cube reports,sent tuesday july error hello guys have receiv...,0,4,21,41,35,1,3,sent tuesday july error hello guys received er...


In [22]:
df = df[['title','description', 'impact']]
df

,title,description,impact
1,connection icon,icon dear please setup icon per icon engineers...,4
2,work experience user,work experience user hi work experience studen...,4
3,requesting meeting,requesting meeting hi please help follow equip...,4
4,reset passwords external accounts,expire days hi ask help update passwords colle...,4
5,mail,verification warning hi got attached please ad...,4
...,...,...,...
48544,git space project issues adding users,sent tuesday july git space hi got rights prob...,3
48545,error,sent july error hi guys help error appearing w...,3
48546,connection issues,sent tuesday july connection issues hello conn...,3
48547,error cube reports,sent tuesday july error hello guys received er...,3


In [25]:
# Combine 'title' and 'description' into a single input
X_text = df['title'].str.lower() + ' ' + df['description'].str.lower()  # Lowercase for normalization

# Text preprocessing (optional)
# You can add steps like removing punctuation, stop words, etc. using libraries like nltk

# Preprocess the text data
max_features = 10000  # Maximum number of words in the vocabulary
max_len = 100  # Maximum length of each ticket text

tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X_text.values)
X = tokenizer.texts_to_sequences(X_text.values)
X = pad_sequences(X, maxlen=max_len)

# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['impact'].values)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
embedding_dim = 128  # Dimension of the word embeddings
lstm_units = 64  # Number of LSTM units
vocab_size = len(tokenizer.word_index) + 1  # Size of the vocabulary

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(units=lstm_units, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=1, activation='sigmoid'))  # Assuming binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy:", scores[1])

# Predict the classes for the test set
y_pred = model.predict(X_test)
y_pred_labels = label_encoder.inverse_transform(argmax(y_pred, axis=-1).flatten())

print("Classification Report:")
print(classification_report(y_test, y_pred_labels))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_labels))

Epoch 1/10


C:\Users\HP\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1196/1196 - 93s - 78ms/step - accuracy: 8.3619e-04 - loss: -1.2791e+02 - val_accuracy: 0.0013 - val_loss: -2.3450e+02
Epoch 2/10
1196/1196 - 85s - 71ms/step - accuracy: 8.3619e-04 - loss: -3.3641e+02 - val_accuracy: 0.0013 - val_loss: -4.4083e+02
Epoch 3/10
1196/1196 - 81s - 67ms/step - accuracy: 8.3619e-04 - loss: -5.4153e+02 - val_accuracy: 0.0013 - val_loss: -6.4647e+02
Epoch 4/10
1196/1196 - 92s - 77ms/step - accuracy: 8.3619e-04 - loss: -7.4615e+02 - val_accuracy: 0.0013 - val_loss: -8.5175e+02
Epoch 5/10
1196/1196 - 76s - 64ms/step - accuracy: 8.3619e-04 - loss: -9.5067e+02 - val_accuracy: 0.0013 - val_loss: -1.0570e+03
Epoch 6/10
1196/1196 - 88s - 73ms/step - accuracy: 8.3619e-04 - loss: -1.1552e+03 - val_accuracy: 0.0013 - val_loss: -1.2624e+03
Epoch 7/10
1196/1196 - 93s - 78ms/step - accuracy: 8.3619e-04 - loss: -1.3599e+03 - val_accuracy: 0.0013 - val_loss: -1.4672e+03
Epoch 8/10
1196/1196 - 110s - 92ms/step - accuracy: 8.3619e-04 - loss: -1.5645e+03 - val_accuracy: 0.0013 - 

NameError: name 'argmax' is not defined

In [29]:
# Combine 'title' and 'description' into a single input
X_text = df['title'].str.lower() + ' ' + df['description'].str.lower()

# Text preprocessing (optional)
# You can add steps like removing punctuation, stop words, etc. using libraries like nltk

# Feature engineering with TF-IDF
vectorizer = TfidfVectorizer(max_features=10000)  # Adjust max_features as needed
X = vectorizer.fit_transform(X_text.values)

# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['impact'].values)

# Check for class imbalance (optional)
class_counts = pd.Series(y).value_counts()
print(f"Class distribution:\n{class_counts}")

# Oversample the minority class (impact class 0) using SMOTE
if class_counts[0] < class_counts[1]:  # Assuming impact class 0 is the minority
    oversample = SMOTE(sampling_strategy='minority')
    X_train, y_train = oversample.fit_resample(X, y)
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the SVM model
model = SVC()  # You can experiment with different kernel types (e.g., 'rbf')

# Train the model
model.fit(X_train, y_train)

# Evaluate the model (using the test set if available)
if len(y_test) > 0:
    y_pred = model.predict(X_test)
    print("Classification Report:")
    print(classification_report(y_test, label_encoder.inverse_transform(y_pred)))
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, label_encoder.inverse_transform(y_pred)))

# Make predictions on new data (optional)
# new_data = ... (your new data to predict on, preprocessed similarly to X)
# new_predictions = model.predict(vectorizer.transform(new_data))
# new_predictions_labels = label_encoder.inverse_transform(new_predictions)

Class distribution:
4    34033
3    13068
0      465
2      227
1       44
Name: count, dtype: int64
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.69      0.80        88
           1       0.00      0.00      0.00        12
           2       1.00      0.02      0.04        54
           3       0.95      0.95      0.95      2529
           4       0.98      0.99      0.99      6885

    accuracy                           0.97      9568
   macro avg       0.77      0.53      0.55      9568
weighted avg       0.97      0.97      0.97      9568

Confusion Matrix:
[[  61    0    0   25    2]
 [   0    0    0   12    0]
 [   0    0    1   48    5]
 [   0    0    0 2408  121]
 [   4    0    0   48 6833]]


D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [30]:
df

,title,description,impact
1,connection icon,icon dear please setup icon per icon engineers...,4
2,work experience user,work experience user hi work experience studen...,4
3,requesting meeting,requesting meeting hi please help follow equip...,4
4,reset passwords external accounts,expire days hi ask help update passwords colle...,4
5,mail,verification warning hi got attached please ad...,4
...,...,...,...
48544,git space project issues adding users,sent tuesday july git space hi got rights prob...,3
48545,error,sent july error hi guys help error appearing w...,3
48546,connection issues,sent tuesday july connection issues hello conn...,3
48547,error cube reports,sent tuesday july error hello guys received er...,3


In [31]:
# Define the replacement dictionary
replacement_dict = {0: 1, 1: 1, 2: 1, 3: 2, 4: 3}

# Map the values in the 'impact' column using the replacement dictionary
df['impact'] = df['impact'].map(replacement_dict)

# Verify the updated counts
print(df['impact'].value_counts())

impact
3    34033
2    13068
1      736
Name: count, dtype: int64


In [33]:

# Combine 'title' and 'description' into a single input
X_text = df['title'].str.lower() + ' ' + df['description'].str.lower()  # Lowercase for normalization

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = vectorizer.fit_transform(X_text)

# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['impact'].values)

# Apply SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define and train SVM model with class weights
class_weight = {0: 1, 1: 2}  # Class weights for class 0 and class 1 respectively
svm_model = SVC(kernel='rbf', gamma='scale', class_weight=class_weight)  # RBF kernel SVM with class weights
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Class distribution:
2    34033
1    13068
0      736
Name: count, dtype: int64
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        88
           1       0.07      1.00      0.14        12
           2       0.00      0.00      0.00        54
           3       0.00      0.01      0.01      2529
           4       0.00      0.00      0.00      6885

    accuracy                           0.00      9568
   macro avg       0.02      0.20      0.03      9568
weighted avg       0.00      0.00      0.00      9568

Confusion Matrix:
[[   0   88    0    0    0]
 [   0   12    0    0    0]
 [   0   54    0    0    0]
 [   0    7 2498   24    0]
 [   0    1    2 6882    0]]


D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Softwares\Jupyter\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
# Combine 'title' and 'description' into a single input
X_text = df['title'].str.lower() + ' ' + df['description'].str.lower()  # Lowercase for normalization

# TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = vectorizer.fit_transform(X_text)

# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['impact'].values)

# Calculate the number of samples needed in class 1 to match class 2
num_samples_class_2 = 34033
num_samples_class_1 = 13068

# Calculate the sampling strategy for SMOTE
sampling_strategy = {1: num_samples_class_2}

# Apply SMOTE to oversample class 1
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_tfidf, y)
print('X resampled', X_resampled)
print('Y resampled', y_resampled)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define and train SVM model
class_weight = {1: 10}  # Class weights for class 1
svm_model = SVC(kernel='rbf', gamma='scale', class_weight=class_weight)  # RBF kernel SVM with class weights
svm_model.fit(X_train, y_train)

# Evaluate the model
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

X resampled   (0, 4925)	0.12682354121097011
  (0, 8928)	0.06713712136881406
  (0, 5726)	0.12168495724218811
  (0, 2461)	0.11268003925148601
  (0, 4978)	0.10193540620018444
  (0, 3007)	0.2278775700007801
  (0, 6316)	0.13777585022936795
  (0, 7987)	0.15130003240745005
  (0, 6485)	0.08254406974781425
  (0, 2264)	0.08331068521224406
  (0, 4223)	0.8985579703805029
  (0, 1862)	0.16294433942084843
  (1, 8924)	0.07013396987729403
  (1, 2801)	0.25948954482555897
  (1, 486)	0.196965395963128
  (1, 5635)	0.16588083882317994
  (1, 5683)	0.1359200881301028
  (1, 5775)	0.15920258631131906
  (1, 1672)	0.2153988129195321
  (1, 8517)	0.33509527294996894
  (1, 4091)	0.0664070363494427
  (1, 9550)	0.290860788026609
  (1, 3296)	0.6400577477275993
  (1, 9912)	0.40030681380600114
  (2, 6491)	0.3160342106140605
  :	:
  (68801, 4740)	0.08959229856699866
  (68801, 6054)	0.06081893691367016
  (68801, 9005)	0.07437789365066433
  (68801, 934)	0.11762743597305395
  (68801, 6017)	0.1317704321510426
  (68801, 439)	0

In [49]:


# Encode the target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['impact'].values)

y

array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [54]:
# Assuming X contains your 'description' and 'title' fields separately and y contains your target variable 'impact'
X_description = df['description']
X_title = df['title']
y = df['impact']

# Split the data into training and testing sets
X_train_desc, X_test_desc, X_train_title, X_test_title, y_train, y_test = train_test_split(X_description, X_title, y, test_size=0.2, random_state=42)

# Initialize TF-IDF vectorizer for 'description'
tfidf_vectorizer_desc = TfidfVectorizer(max_features=10000) 
X_train_tfidf_desc = tfidf_vectorizer_desc.fit_transform(X_train_desc)
X_test_tfidf_desc = tfidf_vectorizer_desc.transform(X_test_desc)

# Initialize TF-IDF vectorizer for 'title'
tfidf_vectorizer_title = TfidfVectorizer(max_features=10000) 
X_train_tfidf_title = tfidf_vectorizer_title.fit_transform(X_train_title)
X_test_tfidf_title = tfidf_vectorizer_title.transform(X_test_title)

# Concatenate the TF-IDF matrices for both features
X_train_tfidf_combined = sp.hstack((X_train_tfidf_desc, X_train_tfidf_title))
X_test_tfidf_combined = sp.hstack((X_test_tfidf_desc, X_test_tfidf_title))

# Calculate the number of samples needed in class 1 to match class 2 and class 3
num_samples_class_1 = sum(y_train == 1)
num_samples_class_2 = sum(y_train == 2)
num_samples_class_3 = sum(y_train == 3)

# Calculate the sampling strategy for Random undersampling for each class
sampling_strategy_undersample = {1: num_samples_class_1, 2: num_samples_class_2, 3: num_samples_class_3}

# Apply Random undersampling to each class separately
undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy_undersample, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_tfidf, y_train)

# Define and train SVM model with class weights
class_weight = {1: 10}  # Class weights for class 1
svm_model = SVC(kernel='rbf', gamma='scale', class_weight=class_weight)  # RBF kernel SVM with class weights
svm_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = svm_model.predict(X_test_tfidf_combined)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

ValueError: X has 13968 features, but SVC is expecting 10000 features as input.